In [ ]:
import json
import torch
import torch.nn as nn
import commentjson
import numpy as np
import os
import gc
import logging  # 파이썬 기본 로깅
from transformers import EarlyStoppingCallback
from transformers import (
    AutoTokenizer,
    AutoModel,
    PreTrainedModel,
    TrainingArguments,
    Trainer
)
# transformers의 로깅을 별도 이름으로 임포트
from transformers import logging as transformers_logging
from datasets import Dataset, Features, Value, Sequence
from sklearn.metrics import accuracy_score, f1_score as sklearn_f1_score
from seqeval.metrics import f1_score, classification_report
from torchcrf import CRF
from tqdm import tqdm
import random
from torch.nn import functional as F

# --- 0. 기본 설정 ---
MODEL_NAME = "klue/roberta-base"  # 모델 크기 증가 (base -> large)
MAX_LEN = 128  # 시퀀스 길이 증가
EPOCHS = 3    # 에폭 수 증가
BATCH_SIZE = 4  # 배치 크기 증가
LEARNING_RATE = 5e-5  # 학습률 최적화
WARMUP_RATIO = 0.1
WEIGHT_DECAY = 0.01
INTENT_WEIGHT = 0.4  # Intent 태스크 가중치 조정
NER_WEIGHT = 0.6     # NER 태스크 가중치 조정 (더 어려운 태스크에 가중치 부여)

def limit_dataset_size(dataset, max_size=1000):
    """데이터셋 크기를 제한하는 함수"""
    if len(dataset) > max_size:
        indices = list(range(len(dataset)))
        random.shuffle(indices)
        indices = indices[:max_size]
        return dataset.select(indices)
    return dataset


# 모델 저장 경로 설정
INTEGRATED_MODEL_DIR = "./models/integrated_nlu_improved"
INTEGRATED_LABEL_PATH = os.path.join(INTEGRATED_MODEL_DIR, "nlu_labels.jsonc")

# Seed 설정 (재현성)
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# 로깅 설정
transformers_logging.set_verbosity_info()  # transformers 로깅 설정
logger = logging.getLogger(__name__)  # 파이썬 기본 로깅 설정

# --- 나머지 코드는 동일 ---

# --- 1. 데이터 로드 함수 ---
def load_intent_data():
    print("인텐트 데이터 로드 중...")
    with open('intent_label_list.jsonc', 'r', encoding='utf-8') as f:
        intent_label_list = commentjson.load(f)

    intent_label_to_id = {label: i for i, label in enumerate(intent_label_list)}
    intent_id_to_label = {i: label for i, label in enumerate(intent_label_list)}

    with open('intent_data.jsonc', 'r', encoding='utf-8') as f:
        intent_data = commentjson.load(f)

    print(f"인텐트 레이블 수: {len(intent_label_list)}")
    print(f"인텐트 데이터 수: {len(intent_data)}")
    return intent_data, intent_label_list, intent_label_to_id, intent_id_to_label

def load_ner_data():
    print("NER 데이터 로드 중...")
    with open('ner_data.jsonc', 'r', encoding='utf-8') as f:
        loaded_ner_data = commentjson.load(f)

    ner_data = []
    for item in loaded_ner_data:
        entities_as_tuples = [tuple(entity_list) for entity_list in item.get("entities", [])]
        ner_data.append({"text": item.get("text", ""), "entities": entities_as_tuples})

    print(f"NER 데이터 수: {len(ner_data)}")
    return ner_data

# --- 2. 통합 NLU 모델 정의 (Intent + NER) - 개선 ---
class ImprovedRobertaForJointIntentAndNER(PreTrainedModel):
    def __init__(self, config, intent_label_to_id, ner_label_to_id):
        super().__init__(config)
        self.num_intent_labels = len(intent_label_to_id)
        self.num_ner_labels = len(ner_label_to_id)

        # 기본 모델 로드
        self.roberta = AutoModel.from_pretrained(MODEL_NAME, config=config)

        # Intent 분류를 위한 헤드 (개선: 더 복잡한 분류기)
        self.intent_dropout = nn.Dropout(0.1)  # 드롭아웃 감소
        self.ner_lstm = nn.LSTM(
            input_size=config.hidden_size,
            hidden_size=config.hidden_size // 2,  # 양방향이므로 절반 크기
            num_layers=2,
            bidirectional=True,
            dropout=0.1,
            batch_first=True
        )
        self.intent_classifier = nn.Linear(config.hidden_size, self.num_intent_labels)  # 단순 선형 분류기

        # NER을 위한 헤드 (개선: BiLSTM + CRF)
        self.ner_dropout = nn.Dropout(0.1)  # 드롭아웃 감소
        self.ner_classifier = nn.Linear(config.hidden_size, self.num_ner_labels)

        # CRF 레이어 (NER용)
        self.crf = CRF(self.num_ner_labels, batch_first=True)

        # 손실 가중치 (두 작업의 균형을 위해 조정)
        self.intent_loss_weight = INTENT_WEIGHT
        self.ner_loss_weight = NER_WEIGHT

        # 가중치 초기화
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        intent_labels=None,
        ner_labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        # RoBERTa 인코더 실행
        outputs = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]  # 모든 토큰의 임베딩
        pooled_output = sequence_output[:, 0, :]  # [CLS] 토큰 임베딩 (Intent 용)

        # Intent 분류 (개선된 분류기)
        intent_output = self.intent_dropout(pooled_output)
        intent_logits = self.intent_classifier(intent_output)

        # NER 분류 (BiLSTM + CRF)
        ner_output = self.ner_dropout(sequence_output)
        ner_lstm_output, _ = self.ner_lstm(ner_output)
        ner_logits = self.ner_classifier(ner_lstm_output)

        # 손실 계산
        total_loss = None
        intent_loss = None
        ner_loss = None

        if intent_labels is not None:
            # Intent 손실 계산 (Label Smoothing 추가)
            intent_loss_fct = nn.CrossEntropyLoss(label_smoothing=0.1)
            intent_loss = intent_loss_fct(intent_logits.view(-1, self.num_intent_labels), intent_labels.view(-1))

        if ner_labels is not None:
            # NER 손실 계산 (CRF)
            mask = attention_mask.bool()

            # -100을 유효한 레이블 인덱스로 변환 (CRF용)
            label_mask = ner_labels != -100
            crf_labels = ner_labels.clone()
            crf_labels[~label_mask] = 0

            # CRF 손실 계산
            ner_loss = -self.crf(ner_logits, crf_labels, mask=mask).mean()

        # 총 손실 계산 (두 작업의 가중치 합)
        if intent_loss is not None and ner_loss is not None:
            total_loss = self.intent_loss_weight * intent_loss + self.ner_loss_weight * ner_loss

        # 예측 단계
        intent_predictions = None
        ner_predictions = None

        if intent_labels is None:
            intent_predictions = torch.argmax(intent_logits, dim=1)

        if ner_labels is None:
            if attention_mask is not None:
                mask = attention_mask.bool()
                best_ner_tags_list = self.crf.decode(ner_logits, mask=mask)

                # 리스트를 텐서로 변환
                ner_predictions = torch.zeros_like(input_ids)
                for i, tags in enumerate(best_ner_tags_list):
                    ner_predictions[i, :len(tags)] = torch.tensor(tags, device=ner_predictions.device)
            else:
                best_ner_tags_list = self.crf.decode(ner_logits)
                ner_predictions = torch.tensor(best_ner_tags_list, device=ner_logits.device)

        return {
            "loss": total_loss,
            "intent_loss": intent_loss,
            "ner_loss": ner_loss,
            "intent_logits": intent_logits,
            "ner_logits": ner_logits,
            "intent_predictions": intent_predictions,
            "ner_predictions": ner_predictions
        }

# --- 3. 데이터 전처리 및 통합 ---
def preprocess_and_merge_data():
    print("\n" + "="*50)
    print("데이터 전처리 및 통합 시작")
    print("="*50)

    # 3.1 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

    # 3.2 Intent 데이터 로드
    intent_data, intent_label_list, intent_label_to_id, intent_id_to_label = load_intent_data()
    print(f"Intent 레이블 ({len(intent_label_list)}개): {intent_label_list}")

    # 3.3 NER 데이터 로드
    ner_data = load_ner_data()

    # 3.4 NER 레이블 정의 (BIO 형식)
    entity_types = set()
    for item in ner_data:
        for _, _, label in item["entities"]:
            entity_types.add(label)
    entity_types = sorted(list(entity_types))

    ner_labels = ["O"]  # Outside tag
    for entity_type in entity_types:
        ner_labels.extend([f"B-{entity_type}", f"I-{entity_type}"])

    ner_label_to_id = {label: i for i, label in enumerate(ner_labels)}
    ner_id_to_label = {i: label for label, i in ner_label_to_id.items()}
    print(f"NER 레이블 ({len(ner_labels)}개): {ner_labels}")

    # 3.5 데이터 통합 및 증강
    integrated_data = []

    # Intent 데이터 전처리 및 추가
    print("\n--- Intent 데이터 전처리 ---")

    # 각 인텐트 클래스별 데이터 수 파악
    intent_class_counts = {}
    for item in intent_data:
        intent_label = None
        possible_keys = ["intent_label", "intent", "label"]

        for key in possible_keys:
            if key in item:
                label_value = item[key]
                if isinstance(label_value, str):
                    intent_label = label_value
                else:
                    intent_label = intent_id_to_label.get(int(label_value), "unknown")
                break

        if intent_label not in intent_class_counts:
            intent_class_counts[intent_label] = 0
        intent_class_counts[intent_label] += 1

    print("인텐트 클래스별 데이터 수:")
    for label, count in intent_class_counts.items():
        print(f"  - {label}: {count}개")

    # Intent 데이터 추가
    for idx, item in enumerate(tqdm(intent_data, desc="인텐트 데이터 처리")):
        try:
            # 텍스트 필드 확인
            if "text" not in item:
                continue

            text = item["text"]

            # 레이블을 찾기 위한 다양한 키 시도
            intent_label = None
            possible_keys = ["intent_label", "intent", "label"]

            for key in possible_keys:
                if key in item:
                    label_value = item[key]
                    # 레이블이 문자열이면 ID로 변환, 이미 숫자면 그대로 사용
                    if isinstance(label_value, str):
                        if label_value in intent_label_to_id:
                            intent_label = intent_label_to_id[label_value]
                        elif label_value.isdigit() and int(label_value) < len(intent_label_list):
                            intent_label = int(label_value)
                    else:  # 숫자인 경우
                        intent_label = int(label_value) if isinstance(label_value, (int, float)) else 0

                    break

            # 레이블을 찾지 못한 경우 기본값 사용
            if intent_label is None:
                intent_label = 0

            # 토큰화
            tokenized = tokenizer(
                text,
                return_offsets_mapping=True,
                add_special_tokens=True,
                truncation=True,
                max_length=MAX_LEN,
                padding='max_length'  # 패딩 추가
            )
            input_ids = tokenized["input_ids"]
            attention_mask = tokenized["attention_mask"]
            offset_mapping = tokenized["offset_mapping"]

            # NER 레이블은 모두 무시 (-100)
            ner_token_labels = [-100] * len(input_ids)

            if idx < 3:  # 샘플 출력
                print(f"\n[Intent 데이터 {idx}] '{text}' -> 의도: {intent_id_to_label.get(intent_label, '알 수 없음')}")

            integrated_data.append({
                "text": text,
                "input_ids": input_ids,
                "attention_mask": attention_mask,
                "intent_label": intent_label,
                "ner_labels": ner_token_labels
            })

            # 데이터 증강: 소량의 노이즈 추가 (랜덤 마스킹)
            if idx % 3 == 0:  # 3개마다 1개씩 증강
                noised_input_ids = input_ids.copy()
                # 10%의 토큰을 랜덤하게 마스킹 또는 다른 토큰으로 대체
                for i in range(1, len(noised_input_ids)-1):  # [CLS]와 [SEP] 토큰은 보존
                    if random.random() < 0.1:  # 10% 확률로 마스킹
                        noised_input_ids[i] = tokenizer.mask_token_id

                integrated_data.append({
                    "text": text + " (augmented)",
                    "input_ids": noised_input_ids,
                    "attention_mask": attention_mask,
                    "intent_label": intent_label,
                    "ner_labels": ner_token_labels
                })

        except Exception as e:
            print(f"Intent 데이터 처리 중 오류 발생 (항목 {idx}): {e}")
            continue

    # NER 데이터 전처리 및 추가
    print("\n--- NER 데이터 전처리 ---")

    # 엔티티 유형별 개수 파악
    entity_type_counts = {}
    for item in ner_data:
        for _, _, entity_type in item["entities"]:
            if entity_type not in entity_type_counts:
                entity_type_counts[entity_type] = 0
            entity_type_counts[entity_type] += 1

    print("엔티티 유형별 개수:")
    for entity_type, count in entity_type_counts.items():
        print(f"  - {entity_type}: {count}개")

    # Intent 간 균형을 위한 인텐트 분포 생성
    intent_counts = {i: 0 for i in range(len(intent_label_list))}
    for item in integrated_data:
        intent_label = item.get("intent_label")
        if intent_label is not None:
            intent_counts[intent_label] += 1

    # 가장 많은 수의 인텐트 ID 찾기
    max_intent_count = max(intent_counts.values())
    min_intent_count = min(intent_counts.values())
    intent_weights = {i: max_intent_count / (count + 1) for i, count in intent_counts.items()}

    # NER 데이터에 인텐트 분포 적용
    for idx, item in enumerate(tqdm(ner_data, desc="NER 데이터 처리")):
        text = item["text"]
        entities = item["entities"]

        # 적절한 인텐트 할당 (매핑 또는 무작위)
        # 의도적으로 부족한 인텐트 클래스에 가중치 부여하여 할당
        weighted_intent_ids = []
        for i, weight in intent_weights.items():
            weighted_intent_ids.extend([i] * int(weight * 10))

        intent_label = random.choice(weighted_intent_ids)

        # 텍스트 기반 인텐트 힌트 (텍스트에 특정 키워드가 있으면 관련 인텐트로 설정)
        # 예: "책, 도서, 대출" -> 도서 관련 인텐트
        # 이 부분은 프로젝트에 맞게 구체적으로 구현 필요

        # 1. 문자 단위 BIO 태깅
        char_labels = ["O"] * len(text)

        # 2. 엔티티에 따라 BIO 태그 할당
        for start_char, end_char, entity_type in entities:
            if start_char < 0:
                start_char = 0
            if end_char > len(text):
                end_char = len(text)

            if start_char < end_char and start_char < len(text):
                for i in range(start_char, end_char):
                    if i == start_char:
                        char_labels[i] = f"B-{entity_type}"
                    else:
                        char_labels[i] = f"I-{entity_type}"

        # 3. 토큰화 및 토큰-문자 정렬
        tokenized = tokenizer(
            text,
            return_offsets_mapping=True,
            add_special_tokens=True,
            truncation=True,
            max_length=MAX_LEN,
            padding='max_length'  # 패딩 추가
        )
        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        offset_mapping = tokenized["offset_mapping"]
        tokens = tokenizer.convert_ids_to_tokens(input_ids)

        # 4. 토큰별 레이블 할당 (개선된 BIO 태깅)
        ner_token_labels = []
        prev_entity_type = None

        for i, (start, end) in enumerate(offset_mapping):
            # 특수 토큰 또는 패딩 토큰 처리
            if start == end or attention_mask[i] == 0:
                token_label = -100  # ignore_index
                ner_token_labels.append(token_label)
                prev_entity_type = None
                continue

            # 토큰에 해당하는 문자 레이블 모음
            token_char_labels = [char_labels[j] for j in range(start, end)]

            # 토큰 내 문자 레이블들 중 B- 태그가 있으면 B- 우선 (한국어 형태소에 적합)
            b_labels = [l for l in token_char_labels if l.startswith("B-")]
            i_labels = [l for l in token_char_labels if l.startswith("I-")]

            if b_labels:
                # B- 태그가 여러 개 있으면 첫 번째 것 사용
                token_label = ner_label_to_id[b_labels[0]]
                prev_entity_type = b_labels[0][2:]  # "B-"를 제외한 엔티티 유형
            elif i_labels:
                # 이전 토큰이 같은 엔티티 유형이었으면 I- 태그 유지
                if prev_entity_type and any(l[2:] == prev_entity_type for l in i_labels):
                    token_label = ner_label_to_id[f"I-{prev_entity_type}"]
                else:
                    # 그렇지 않으면 새로운 엔티티 시작으로 취급 (B- 태그로 변환)
                    entity_type = i_labels[0][2:]  # "I-"를 제외한 엔티티 유형
                    token_label = ner_label_to_id[f"B-{entity_type}"]
                    prev_entity_type = entity_type
            else:
                token_label = ner_label_to_id["O"]
                prev_entity_type = None

            ner_token_labels.append(token_label)

        # 레이블 길이 확인 및 조정
        if len(ner_token_labels) < len(input_ids):
            ner_token_labels.extend([-100] * (len(input_ids) - len(ner_token_labels)))
        elif len(ner_token_labels) > len(input_ids):
            ner_token_labels = ner_token_labels[:len(input_ids)]

        if idx < 3:  # 샘플 출력
            print(f"\n[NER 데이터 {idx}] '{text}' -> 엔티티: {entities}")
            print(f"  인텐트: {intent_id_to_label.get(intent_label, '알 수 없음')}")
            print(f"  토큰화: {tokens[:10]}... (총 {len(tokens)}개)")
            print(f"  NER 레이블: {[ner_id_to_label.get(l, 'IGN') for l in ner_token_labels[:10]]}... (총 {len(ner_token_labels)}개)")

        integrated_data.append({
            "text": text,
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "intent_label": intent_label,
            "ner_labels": ner_token_labels
        })

        # 데이터 증강: 엔티티가 많은 데이터는 중복해서 추가
        num_entities = len(entities)
        if num_entities > 2:  # 엔티티가 3개 이상이면 중복 추가
            integrated_data.append({
                "text": text + " (duplicated)",
                "input_ids": input_ids,
                "attention_mask": attention_mask,
                "intent_label": intent_label,
                "ner_labels": ner_token_labels
            })

    print(f"\n통합 데이터셋 크기: {len(integrated_data)}")

    # 3.6 데이터셋 생성
    integrated_features = Features({
        'text': Value('string'),
        'input_ids': Sequence(Value('int32'), length=MAX_LEN),
        'attention_mask': Sequence(Value('int32'), length=MAX_LEN),
        'intent_label': Value('int32'),
        'ner_labels': Sequence(Value('int32'), length=MAX_LEN)
    })

    integrated_dataset = Dataset.from_list(integrated_data, features=integrated_features)

    # 학습/검증/테스트 데이터셋 분리
    train_val_test_datasets = integrated_dataset.train_test_split(test_size=0.3, seed=SEED)
    train_dataset = train_val_test_datasets["train"]

    # 테스트 데이터를 검증과 테스트로 분리
    test_valid_datasets = train_val_test_datasets["test"].train_test_split(test_size=0.5, seed=SEED)
    valid_dataset = test_valid_datasets["train"]
    test_dataset = test_valid_datasets["test"]

    train_dataset = limit_dataset_size(train_dataset, max_size=1000)
    valid_dataset = limit_dataset_size(valid_dataset, max_size=300)
    test_dataset = limit_dataset_size(test_dataset, max_size=300)

    print(f"훈련 데이터 크기: {len(train_dataset)}")
    print(f"검증 데이터 크기: {len(valid_dataset)}")
    print(f"테스트 데이터 크기: {len(test_dataset)}")

    return train_dataset, valid_dataset, test_dataset, intent_label_to_id, intent_id_to_label, ner_label_to_id, ner_id_to_label

# --- 4. 데이터 콜레이터 정의 ---
class ImprovedJointNLUDataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.pad_token_id = tokenizer.pad_token_id

    def __call__(self, features):
        # 입력 시퀀스 최대 길이 계산
        max_length = max(len(feature["input_ids"]) for feature in features)
        max_length = min(max_length, MAX_LEN)  # 최대 길이 제한

        # 배치 준비
        batch = {
            "input_ids": [],
            "attention_mask": [],
            "intent_labels": [],
            "ner_labels": []
        }

        for feature in features:
            # 패딩 적용
            input_ids = feature["input_ids"]
            attention_mask = feature["attention_mask"]
            ner_labels = feature["ner_labels"]

            # 길이 확인 및 조정
            if len(input_ids) > max_length:
                input_ids = input_ids[:max_length]
                attention_mask = attention_mask[:max_length]
                ner_labels = ner_labels[:max_length]

            padding_length = max_length - len(input_ids)

            # 입력 패딩
            if padding_length > 0:
                input_ids = input_ids + [self.pad_token_id] * padding_length
                attention_mask = attention_mask + [0] * padding_length
                ner_labels = ner_labels + [-100] * padding_length  # -100은 손실 계산에서 무시됨

            # 배치에 추가
            batch["input_ids"].append(input_ids)
            batch["attention_mask"].append(attention_mask)
            batch["intent_labels"].append(feature.get("intent_label", 0))
            batch["ner_labels"].append(ner_labels)

        # 텐서로 변환
        batch["input_ids"] = torch.tensor(batch["input_ids"], dtype=torch.long)
        batch["attention_mask"] = torch.tensor(batch["attention_mask"], dtype=torch.long)
        batch["intent_labels"] = torch.tensor(batch["intent_labels"], dtype=torch.long)
        batch["ner_labels"] = torch.tensor(batch["ner_labels"], dtype=torch.long)

        return batch

# --- 5. 개선된 커스텀 트레이너 정의 (통합 NLU용) ---
class ImprovedJointNLUTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.best_metrics = {'joint_score': 0}

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # 안정적인 손실 계산을 위한 예외 처리 추가
        try:
            outputs = model(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                intent_labels=inputs["intent_labels"],
                ner_labels=inputs["ner_labels"]
            )

            loss = outputs["loss"]
            intent_loss = outputs["intent_loss"]
            ner_loss = outputs["ner_loss"]

            # 손실 값 검증 (NaN 방지)
            if torch.isnan(loss) or torch.isinf(loss):
                logger.warning("발견된 NaN/Inf 손실값! 대체값 사용...")
                loss = torch.tensor(1.0, device=loss.device, requires_grad=True)

            # 자세한 로깅 (정기적으로)
            if self.state.global_step % 10 == 0:
                self.log({
                    "total_loss": loss.item(),
                    "intent_loss": intent_loss.item() if intent_loss is not None else 0,
                    "ner_loss": ner_loss.item() if ner_loss is not None else 0,
                    "lr": self.optimizer.param_groups[0]['lr']  # 현재 학습률 로깅
                })

            return (loss, outputs) if return_outputs else loss

        except Exception as e:
            logger.error(f"손실 계산 중 오류 발생: {e}")
            # 오류 시 기본 손실 반환
            loss = torch.tensor(1.0, device=inputs["input_ids"].device, requires_grad=True)
            return (loss, None) if return_outputs else loss

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        with torch.no_grad():
            # 입력 데이터 검증
            for k, v in inputs.items():
                if torch.isnan(v).any() or torch.isinf(v).any():
                    logger.warning(f"입력 데이터 문제 발견 ({k})!")
                    inputs[k] = torch.nan_to_num(v)  # NaN/Inf 값 대체

            outputs = model(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                intent_labels=inputs["intent_labels"],
                ner_labels=inputs["ner_labels"]
            )

            loss = outputs["loss"]

            if prediction_loss_only:
                return (loss, None, None)

            # 예측 및 레이블 추출
            intent_logits = outputs["intent_logits"]
            intent_predictions = torch.argmax(intent_logits, dim=1)
            intent_labels = inputs["intent_labels"]

            # CRF 기반 NER 예측 처리
            ner_logits = outputs["ner_logits"]
            attention_mask = inputs["attention_mask"].bool()
            ner_predictions = torch.zeros_like(inputs["input_ids"])

            try:
                # CRF 디코딩 (최적의 태그 시퀀스 찾기)
                best_ner_tags_list = model.crf.decode(ner_logits, mask=attention_mask)

                for i, tags in enumerate(best_ner_tags_list):
                    length = min(len(tags), ner_predictions.size(1))
                    ner_predictions[i, :length] = torch.tensor(tags[:length], device=ner_predictions.device)
            except Exception as e:
                logger.error(f"NER 예측 중 오류 발생: {e}")
                # 오류 시 모든 토큰을 'O'(Outside) 태그로 예측
                ner_predictions = torch.zeros_like(inputs["input_ids"])

            ner_labels = inputs["ner_labels"]

            # 반환값: (손실, 예측값 튜플, 레이블 튜플)
            return (loss, (intent_predictions, ner_predictions), (intent_labels, ner_labels))

    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
        """향상된 평가 메서드: 점진적 평가 및 상세 메트릭스"""
        metrics = super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)

        # 개선된 결과 저장 로직 (새로운 최고 성능 달성 시)
        current_score = metrics.get(f"{metric_key_prefix}_joint_score", 0)
        if current_score > self.best_metrics['joint_score']:
            self.best_metrics['joint_score'] = current_score
            self.best_metrics['step'] = self.state.global_step
            self.best_metrics['epoch'] = self.state.epoch

            # 상세 로깅
            logger.info(f"새로운 최고 성능 달성! Joint Score: {current_score:.4f} (Step: {self.state.global_step}, Epoch: {self.state.epoch:.2f})")

        return metrics

# --- 6. 개선된 평가 지표 계산 함수 ---
def compute_improved_joint_metrics(eval_preds, ner_label_list=None):
    (intent_preds, ner_preds), (intent_labels, ner_labels) = eval_preds

    # Intent 평가 (F1 스코어 추가)
    intent_accuracy = accuracy_score(intent_labels, intent_preds)
    intent_f1_macro = sklearn_f1_score(intent_labels, intent_preds, average='macro')
    intent_f1_weighted = sklearn_f1_score(intent_labels, intent_preds, average='weighted')

    # NER 평가 준비
    true_ner_predictions = []
    true_ner_labels = []

    for prediction, label, mask in zip(ner_preds, ner_labels, attention_mask if 'attention_mask' in eval_preds else None):
        true_pred = []
        true_label = []

        for p, l in zip(prediction, label):
            if l != -100:  # -100은 무시
                # NER 레이블 ID를 문자열 레이블로 변환
                try:
                    true_pred.append(ner_id_to_label[p.item()])
                    true_label.append(ner_id_to_label[l.item()])
                except Exception as e:
                    logger.error(f"레이블 변환 오류: {e} - p={p.item()}, l={l.item()}")
                    true_pred.append("O")  # 오류 시 기본값
                    true_label.append("O")

        true_ner_predictions.append(true_pred)
        true_ner_labels.append(true_label)

    # seqeval의 f1_score 계산 (더 상세한 메트릭스)
    try:
        ner_f1 = f1_score(true_ner_labels, true_ner_predictions)
        ner_report = classification_report(true_ner_labels, true_ner_predictions, digits=4, output_dict=True)

        # 개체 유형별 F1 점수 추출
        entity_metrics = {}
        for label, metrics in ner_report.items():
            if label not in ['micro avg', 'macro avg', 'weighted avg'] and isinstance(metrics, dict):
                entity_metrics[f"ner_f1_{label}"] = metrics.get('f1-score', 0)

        # 평가 보고서 출력 (텍스트 형식)
        print("\nNER Classification Report:\n", classification_report(true_ner_labels, true_ner_predictions, digits=4))

        # 통합 점수 계산 (가중 평균, NER에 더 높은 가중치)
        joint_score = INTENT_WEIGHT * intent_accuracy + NER_WEIGHT * ner_f1

        # 최종 메트릭스 반환 (상세한 지표 포함)
        metrics = {
            "intent_accuracy": intent_accuracy,
            "intent_f1_macro": intent_f1_macro,
            "intent_f1_weighted": intent_f1_weighted,
            "ner_f1": ner_f1,
            "joint_score": joint_score,
            **entity_metrics  # 개체 유형별 성능 추가
        }

        return metrics

    except Exception as e:
        logger.error(f"평가 지표 계산 오류: {e}")
        return {
            "intent_accuracy": intent_accuracy,
            "intent_f1_macro": intent_f1_macro,
            "intent_f1_weighted": intent_f1_weighted,
            "ner_f1": 0.0,
            "joint_score": intent_accuracy * INTENT_WEIGHT
        }

# --- 7. 학습률 스케줄러 개선 ---
def get_linear_schedule_with_warmup_and_decay(optimizer, num_warmup_steps, num_training_steps, min_lr_ratio=0.1):
    """학습률 워밍업 및 선형 감소 스케줄러"""

    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            # 워밍업 구간
            return float(current_step) / float(max(1, num_warmup_steps))

        # 워밍업 이후 선형 감소
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return max(min_lr_ratio, 1.0 - progress)

    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

# --- 8. 개선된 통합 NLU 모델 훈련 함수 ---
def train_improved_integrated_nlu_model():
    print("\n" + "="*50)
    print("개선된 통합 NLU 모델 훈련 시작 (Intent + NER)")
    print("="*50)

    # 글로벌 변수 설정 (평가 지표용)
    global intent_id_to_label, ner_id_to_label, tokenizer

    # 8.1 데이터 전처리 및 통합
    train_dataset, valid_dataset, test_dataset, intent_label_to_id, intent_id_to_label, ner_label_to_id, ner_id_to_label = preprocess_and_merge_data()

    # 8.2 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

    # 8.3 개선된 데이터 콜레이터 초기화
    data_collator = ImprovedJointNLUDataCollator(tokenizer)

    # 8.4 모델 설정
    config = AutoModel.from_pretrained(MODEL_NAME).config
    config.intent_label_to_id = intent_label_to_id
    config.ner_label_to_id = ner_label_to_id

    # 8.5 개선된 통합 NLU 모델 초기화
    model = ImprovedRobertaForJointIntentAndNER(
        config,
        intent_label_to_id,
        ner_label_to_id
    )

    # 8.6 개선된 훈련 설정
    training_args = TrainingArguments(
        output_dir="./results/improved_integrated_nlu",
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=1,  # 그래디언트 누적 단순화
        logging_dir='./logs/improved_integrated_nlu',
        logging_steps=10,
        save_steps=100,  # 저장 빈도 감소
        eval_steps=100,
        eval_strategy="steps",
        load_best_model_at_end=True,
        save_total_limit=1,  # 최고 성능 모델 더 많이 저장
        fp16=True,  # 혼합 정밀도 훈련 활성화 (속도 향상)
        greater_is_better=True,
        metric_for_best_model="intent_accuracy",
        weight_decay=WEIGHT_DECAY,
        learning_rate=LEARNING_RATE,
        warmup_ratio=WARMUP_RATIO,
        lr_scheduler_type="linear",  # 개선된 스케줄러
        report_to="none"
    )

    # 8.7 개선된 훈련 시작
    trainer = ImprovedJointNLUTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=data_collator,
        compute_metrics=compute_improved_joint_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # 메모리 최적화
    gc.collect()
    torch.cuda.empty_cache() if torch.cuda.is_available() else None

    # 훈련 실행
    print("모델 훈련 시작...")
    trainer.train()

    # 8.8 최종 평가
    print("\n--- 검증 세트 최종 평가 ---")
    eval_result = trainer.evaluate(valid_dataset)
    print(f"검증 세트 평가 결과: {eval_result}")

    print("\n--- 테스트 세트 최종 평가 ---")
    test_result = trainer.evaluate(test_dataset, metric_key_prefix="test")
    print(f"테스트 세트 평가 결과: {test_result}")

    # 8.9 모델 및 레이블 정보 저장
    print("\n모델 및 레이블 정보 저장 중...")
    os.makedirs(INTEGRATED_MODEL_DIR, exist_ok=True)

    # 메모리 최적화
    gc.collect()
    model.cpu()

    # 모델 저장
    torch.save(model.state_dict(), os.path.join(INTEGRATED_MODEL_DIR, "pytorch_model.bin"))

    # Config 정보 저장
    config_dict = config.to_dict()
    config_dict["model_type"] = "improved_roberta_joint_nlu"  # 개선된 모델 타입 표시
    with open(os.path.join(INTEGRATED_MODEL_DIR, "config.json"), "w", encoding="utf-8") as f:
        json.dump(config_dict, f, ensure_ascii=False, indent=4)

    # 토크나이저 저장
    tokenizer.save_pretrained(INTEGRATED_MODEL_DIR)

    # 레이블 매핑 저장
    with open(INTEGRATED_LABEL_PATH, 'w', encoding='utf-8') as f:
        json.dump({
            "intent_id2label": intent_id_to_label,
            "intent_label2id": intent_label_to_id,
            "ner_id2label": ner_id_to_label,
            "ner_label2id": ner_label_to_id,
            "training_metrics": {
                "validation": eval_result,
                "test": test_result,
                "best_metrics": trainer.best_metrics
            }
        }, f, ensure_ascii=False, indent=2)

    print(f"개선된 통합 NLU 모델 및 레이블 정보 저장 완료: {INTEGRATED_MODEL_DIR}")
    print("개선된 통합 NLU 모델 훈련 완료!")

# --- 9. 메인 실행 코드 ---
if __name__ == "__main__":
    try:
        # 시작 시간 기록
        import time
        start_time = time.time()

        # 개선된 훈련 함수 실행
        train_improved_integrated_nlu_model()

        # 총 실행 시간 출력
        total_time = time.time() - start_time
        hours, remainder = divmod(total_time, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f"\n총 실행 시간: {int(hours)}시간 {int(minutes)}분 {seconds:.2f}초")

    except Exception as e:
        print(f"모델 훈련 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()


개선된 통합 NLU 모델 훈련 시작 (Intent + NER)

데이터 전처리 및 통합 시작


loading file vocab.txt from cache at C:\Users\JA104\.cache\huggingface\hub\models--klue--roberta-base\snapshots\02f94ba5e3fcb7e2a58a390b8639b0fac974a8da\vocab.txt
loading file tokenizer.json from cache at C:\Users\JA104\.cache\huggingface\hub\models--klue--roberta-base\snapshots\02f94ba5e3fcb7e2a58a390b8639b0fac974a8da\tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at C:\Users\JA104\.cache\huggingface\hub\models--klue--roberta-base\snapshots\02f94ba5e3fcb7e2a58a390b8639b0fac974a8da\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\JA104\.cache\huggingface\hub\models--klue--roberta-base\snapshots\02f94ba5e3fcb7e2a58a390b8639b0fac974a8da\tokenizer_config.json
loading file chat_template.jinja from cache at None


인텐트 데이터 로드 중...
인텐트 레이블 수: 40
인텐트 데이터 수: 1371
Intent 레이블 (40개): ['search_book_title', 'search_book_author', 'search_book_location', 'check_book_availability', 'get_bestseller', 'get_new_releases', 'request_recommendation_genre', 'request_recommendation_mood', 'request_recommendation_topic', 'request_recommendation_similar', 'request_recommendation_reader', 'search_space_availability', 'reserve_space', 'get_space_info', 'check_space_reservation', 'cancel_space_reservation', 'search_program', 'apply_program', 'get_program_info', 'check_program_application', 'cancel_program_application', 'get_library_hours', 'inquire_service', 'manage_membership', 'check_loan_status', 'extend_loan', 'reserve_book', 'check_reservation_status', 'cancel_book_reservation', 'check_overdue_status', 'report_lost_item', 'greeting', 'gratitude', 'closing', 'affirmative', 'negative', 'abuse', 'clarification', 'out_of_scope', 'repeat']
NER 데이터 로드 중...
NER 데이터 수: 1425
NER 레이블 (51개): ['O', 'B-account_action', 'I-accou

인텐트 데이터 처리:  69%|██████▉   | 949/1371 [00:00<00:00, 4914.00it/s]


[Intent 데이터 0] '나루토 책 있어?' -> 의도: search_book_title

[Intent 데이터 1] '식물학자의 숲속 일기라는 책 있나요?' -> 의도: search_book_title

[Intent 데이터 2] '사랑의 기술 책 찾아주세요.' -> 의도: search_book_title


...

NER 데이터 처리: 100%|██████████| 1425/1425 [00:00<00:00, 2488.91it/s]



통합 데이터셋 크기: 3363
훈련 데이터 크기: 500
검증 데이터 크기: 100
테스트 데이터 크기: 100


loading file vocab.txt from cache at C:\Users\JA104\.cache\huggingface\hub\models--klue--roberta-base\snapshots\02f94ba5e3fcb7e2a58a390b8639b0fac974a8da\vocab.txt
loading file tokenizer.json from cache at C:\Users\JA104\.cache\huggingface\hub\models--klue--roberta-base\snapshots\02f94ba5e3fcb7e2a58a390b8639b0fac974a8da\tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at C:\Users\JA104\.cache\huggingface\hub\models--klue--roberta-base\snapshots\02f94ba5e3fcb7e2a58a390b8639b0fac974a8da\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\JA104\.cache\huggingface\hub\models--klue--roberta-base\snapshots\02f94ba5e3fcb7e2a58a390b8639b0fac974a8da\tokenizer_config.json
loading file chat_template.jinja from cache at None
loading configuration file config.json from cache at C:\Users\JA104\.cache\huggingface\hub\models--klue--roberta-base\snapshots\02f94ba5e3fcb7e2a58a390b8639b0fac974a8da\config.json
Mod

모델 훈련 시작...


: 